In [1]:
import pandas as pd
import numpy as np
import re
import json
import ast

As we are still unsure of the data we want, and are subject to changing our minds, I will be dropping irrelevant columns at the very end. This will allow us to quickly re-add columns in case we do change our minds.

In [2]:
#reading in activity watch data

activity_watch = pd.read_json("activity_watch_2021_04_18.json")


In [3]:
#separating the buckets column as that is where the dictionary of relevant data is

activity_watch = activity_watch['buckets'].apply(pd.Series)
activity_watch

,id,created,name,type,client,hostname,events
aw-stopwatch,aw-stopwatch,2021-02-27T07:11:34.038428+00:00,None,general.stopwatch,aw-webui,unknown,[]
aw-watcher-afk_DESKTOP-TOACMEE,aw-watcher-afk_DESKTOP-TOACMEE,2021-02-27T07:07:20.676248+00:00,None,afkstatus,aw-watcher-afk,DESKTOP-TOACMEE,[{'timestamp': '2021-04-18T09:41:13.362000+00:...
aw-watcher-window_DESKTOP-TOACMEE,aw-watcher-window_DESKTOP-TOACMEE,2021-02-27T07:07:20.499312+00:00,None,currentwindow,aw-watcher-window,DESKTOP-TOACMEE,[{'timestamp': '2021-04-18T14:15:21.485000+00:...


In [4]:
#Further splitting the events column to extract the time stamp, duration and data

activity_watch_split = pd.DataFrame(activity_watch['events'][2])
activity_watch_split

,timestamp,duration,data
0,2021-04-18T14:15:21.485000+00:00,10.812,"{'app': 'msedge.exe', 'title': 'ActivityWatch ..."
1,2021-04-18T14:15:20.283000+00:00,0.000,"{'app': 'aw-qt.exe', 'title': 'aw-qt'}"
2,2021-04-18T14:15:17.916000+00:00,1.196,"{'app': 'explorer.exe', 'title': ''}"
3,2021-04-18T14:15:16.716000+00:00,0.000,"{'app': 'msedge.exe', 'title': 'Oura API and 7..."
4,2021-04-18T14:15:15.525000+00:00,0.000,"{'app': 'msedge.exe', 'title': 'localhost:7777..."
...,...,...,...
26718,2021-02-27T15:09:10.151000+00:00,4.846,"{'app': 'SearchApp.exe', 'title': 'Search'}"
26719,2021-02-27T15:08:49.183000+00:00,19.718,"{'app': 'msedge.exe', 'title': 'Sponsor @Activ..."
26720,2021-02-27T15:08:24.630000+00:00,23.357,"{'app': 'msedge.exe', 'title': 'ActivityWatch ..."
26721,2021-02-27T15:08:23.428000+00:00,0.000,"{'app': 'msedge.exe', 'title': 'https://github..."


In [5]:
#dropping the data column in a copy of this

activity_watch_split_1 = activity_watch_split.drop(columns=["data"])
activity_watch_split_1

,timestamp,duration
0,2021-04-18T14:15:21.485000+00:00,10.812
1,2021-04-18T14:15:20.283000+00:00,0.000
2,2021-04-18T14:15:17.916000+00:00,1.196
3,2021-04-18T14:15:16.716000+00:00,0.000
4,2021-04-18T14:15:15.525000+00:00,0.000
...,...,...
26718,2021-02-27T15:09:10.151000+00:00,4.846
26719,2021-02-27T15:08:49.183000+00:00,19.718
26720,2021-02-27T15:08:24.630000+00:00,23.357
26721,2021-02-27T15:08:23.428000+00:00,0.000


In [6]:
#splitting the data column of activity_watch_split, to extract the app and title being used

activity_watch_split_2 = activity_watch_split['data'].apply(pd.Series)
activity_watch_split_2

,app,title
0,msedge.exe,ActivityWatch and 79 more pages - Personal - M...
1,aw-qt.exe,aw-qt
2,explorer.exe,
3,msedge.exe,Oura API and 78 more pages - Personal - Micros...
4,msedge.exe,localhost:7777/callback?code=AQAAFYrVKDTnt6XUV...
...,...,...
26718,SearchApp.exe,Search
26719,msedge.exe,Sponsor @ActivityWatch on GitHub Sponsors and ...
26720,msedge.exe,ActivityWatch and 22 more pages - Personal - M...
26721,msedge.exe,https://github.com/ActivityWatch and 22 more p...


In [126]:
#combining the two split activity watch dataframes so that it contains both the timestamp, app, title and duration

activity_watch_final = pd.concat([activity_watch_split_1, activity_watch_split_2], axis=1)
activity_watch_final

,timestamp,duration,app,title
0,2021-04-18T14:15:21.485000+00:00,10.812,msedge.exe,ActivityWatch and 79 more pages - Personal - M...
1,2021-04-18T14:15:20.283000+00:00,0.000,aw-qt.exe,aw-qt
2,2021-04-18T14:15:17.916000+00:00,1.196,explorer.exe,
3,2021-04-18T14:15:16.716000+00:00,0.000,msedge.exe,Oura API and 78 more pages - Personal - Micros...
4,2021-04-18T14:15:15.525000+00:00,0.000,msedge.exe,localhost:7777/callback?code=AQAAFYrVKDTnt6XUV...
...,...,...,...,...
26718,2021-02-27T15:09:10.151000+00:00,4.846,SearchApp.exe,Search
26719,2021-02-27T15:08:49.183000+00:00,19.718,msedge.exe,Sponsor @ActivityWatch on GitHub Sponsors and ...
26720,2021-02-27T15:08:24.630000+00:00,23.357,msedge.exe,ActivityWatch and 22 more pages - Personal - M...
26721,2021-02-27T15:08:23.428000+00:00,0.000,msedge.exe,https://github.com/ActivityWatch and 22 more p...


Here I will be checking for any duplicates in the timestamp column, and any anomalies in the duration column. I will also be going over any nulls in the app and title column.

In [127]:
#checking for nulls in the title column that may not have been labelled as null

activity_watch_nulls = activity_watch_final[activity_watch_final['title'] == ""]
activity_watch_nulls


,timestamp,duration,app,title
2,2021-04-18T14:15:17.916000+00:00,1.196,explorer.exe,
60,2021-04-18T14:04:11.987000+00:00,0.000,explorer.exe,
85,2021-04-18T14:00:06.266000+00:00,0.000,explorer.exe,
87,2021-04-18T14:00:00.310000+00:00,2.361,explorer.exe,
129,2021-04-18T13:47:31.655000+00:00,0.000,explorer.exe,
...,...,...,...,...
26648,2021-02-27T15:23:45.428000+00:00,0.100,explorer.exe,
26684,2021-02-27T15:15:53.395000+00:00,0.000,explorer.exe,
26688,2021-02-27T15:15:44.524000+00:00,0.027,explorer.exe,
26692,2021-02-27T15:15:38.558000+00:00,0.042,explorer.exe,


In [128]:
#checking if these null columns have any useful value in the app column

activity_watch_nulls["app"].value_counts()

explorer.exe                                                      1430
unknown                                                            479
msedge.exe                                                         110
msrdc.exe                                                           48
CredentialUIBroker.exe                                              14
LockApp.exe                                                         11
dwm.exe                                                              8
Zoom.exe                                                             5
Code - Insiders.exe                                                  3
WINWORD.EXE                                                          2
Code.exe                                                             2
python.exe                                                           2
ScreenClippingHost.exe                                               2
OpenWith.exe                                                         2
rundll

The majority of it is unlabelled browsing on internet browser. We have two options:

Option 1) Drop all these empty columns
Option 2) Keep these empty columns

This will depend on how much information the original "app" column gives us, so I will be checking that first.

In [129]:
activity_watch_final["app"].value_counts()[0:15]

msedge.exe                  15235
explorer.exe                 2907
Code.exe                     1656
unknown                      1437
msrdc.exe                    1304
Code - Insiders.exe          1190
LockApp.exe                   573
notepad.exe                   372
msrdcw.exe                    269
ApplicationFrameHost.exe      258
WindowsTerminal.exe           240
ShellExperienceHost.exe       213
Zoom.exe                      181
Spotify.exe                   140
CredentialUIBroker.exe        140
Name: app, dtype: int64

Given that the majority of apps are msedge and explorer, and the null "app" values represent 9% and 16% of the original "app" column, it is best that we keep this data, as they are statistically significant.

Now we will be checking for duplicates in the timestamp column.

In [130]:
#extracting just the duplicates in the timestamp column

activity_watch_final_duplicates = activity_watch_final[activity_watch_final.duplicated(subset=["timestamp"]) == True]
activity_watch_final_duplicates.duplicated(subset=["timestamp"]).value_counts()



False    437
True      57
dtype: int64

In [131]:
#diving deeper into the duplicates to see if other variables such as "app" and "title" are the same

activity_watch_final_duplicates[30:51]

,timestamp,duration,app,title
19361,2021-03-14T14:20:57.804000+00:00,311.013,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19362,2021-03-14T14:20:57.804000+00:00,300.541,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19363,2021-03-14T14:20:57.804000+00:00,290.180,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19364,2021-03-14T14:20:57.804000+00:00,279.835,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19365,2021-03-14T14:20:57.804000+00:00,269.534,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19366,2021-03-14T14:20:57.804000+00:00,259.147,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19367,2021-03-14T14:20:57.804000+00:00,248.822,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19368,2021-03-14T14:20:57.804000+00:00,238.499,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19369,2021-03-14T14:20:57.804000+00:00,228.142,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19370,2021-03-14T14:20:57.804000+00:00,217.765,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...


In [132]:
#taking only the maximum value duplicate

activity_watch_final = activity_watch_final.sort_values('duration').drop_duplicates('timestamp', keep='last')
activity_watch_final

,timestamp,duration,app,title
9520,2021-04-05T13:36:32.387000+00:00,0.000,unknown,unknown
21612,2021-03-11T02:41:42.980000+00:00,0.000,explorer.exe,
11721,2021-03-31T14:39:21.656000+00:00,0.000,msedge.exe,bc hydro - transfer account - residential only...
4882,2021-04-15T05:26:09.626000+00:00,0.000,msedge.exe,New tab and 17 more pages - Personal - Microso...
11724,2021-03-31T14:38:54.668000+00:00,0.000,msedge.exe,WhatsApp Web and 11 more pages - Personal - Mi...
...,...,...,...,...
17935,2021-03-15T04:57:50.661000+00:00,36343.626,unknown,
13081,2021-03-27T02:29:07.006000+00:00,42341.804,LockApp.exe,Windows Default Lock Screen
13334,2021-03-25T23:30:00.056000+00:00,48138.768,LockApp.exe,Windows Default Lock Screen
11910,2021-03-30T23:44:12.699000+00:00,50594.155,LockApp.exe,Windows Default Lock Screen


In [133]:
#renaming duration and timestamp column to reflect the appropriate units

activity_watch_final.rename(columns={"duration":"duration_seconds"}, inplace=True)
activity_watch_final.rename(columns={"timestamp":"timestamp_utc"}, inplace=True)
activity_watch_final


,timestamp_utc,duration_seconds,app,title
9520,2021-04-05T13:36:32.387000+00:00,0.000,unknown,unknown
21612,2021-03-11T02:41:42.980000+00:00,0.000,explorer.exe,
11721,2021-03-31T14:39:21.656000+00:00,0.000,msedge.exe,bc hydro - transfer account - residential only...
4882,2021-04-15T05:26:09.626000+00:00,0.000,msedge.exe,New tab and 17 more pages - Personal - Microso...
11724,2021-03-31T14:38:54.668000+00:00,0.000,msedge.exe,WhatsApp Web and 11 more pages - Personal - Mi...
...,...,...,...,...
17935,2021-03-15T04:57:50.661000+00:00,36343.626,unknown,
13081,2021-03-27T02:29:07.006000+00:00,42341.804,LockApp.exe,Windows Default Lock Screen
13334,2021-03-25T23:30:00.056000+00:00,48138.768,LockApp.exe,Windows Default Lock Screen
11910,2021-03-30T23:44:12.699000+00:00,50594.155,LockApp.exe,Windows Default Lock Screen


In [134]:
#checking for numerical anomalies

activity_watch_final["duration_seconds"].describe()


count    26229.000000
mean        91.257934
std        975.146928
min          0.000000
25%          0.000000
50%          3.441000
75%         14.448000
max      53778.259000
Name: duration_seconds, dtype: float64

In [135]:
#checking for numerical anomalies that look too small

activity_watch_final[activity_watch_final["duration_seconds"] == 0]


,timestamp_utc,duration_seconds,app,title
9520,2021-04-05T13:36:32.387000+00:00,0.0,unknown,unknown
21612,2021-03-11T02:41:42.980000+00:00,0.0,explorer.exe,
11721,2021-03-31T14:39:21.656000+00:00,0.0,msedge.exe,bc hydro - transfer account - residential only...
4882,2021-04-15T05:26:09.626000+00:00,0.0,msedge.exe,New tab and 17 more pages - Personal - Microso...
11724,2021-03-31T14:38:54.668000+00:00,0.0,msedge.exe,WhatsApp Web and 11 more pages - Personal - Mi...
...,...,...,...,...
19627,2021-03-13T21:36:04.150000+00:00,0.0,Code.exe,utils.py - eeg-notebooks - Visual Studio Code
15814,2021-03-20T20:56:14.954000+00:00,0.0,Code - Insiders.exe,gpt.py - braindump - Visual Studio Code - Insi...
2256,2021-04-17T19:34:22.748000+00:00,0.0,Code.exe,research.ts - neurosity-research-program-vscod...
19630,2021-03-13T21:35:00.531000+00:00,0.0,EXCEL.EXE,Find and Replace


In [136]:
#checking for numerical anomalies that look too big

activity_watch_final[activity_watch_final["duration_seconds"] > 14].sort_values("duration_seconds", ascending=False)[30:50]

,timestamp_utc,duration_seconds,app,title
8058,2021-04-10T00:35:38.425000+00:00,10773.160,unknown,unknown
23762,2021-03-03T00:52:05.165000+00:00,10194.149,unknown,
8738,2021-04-06T16:32:58.208000+00:00,9550.460,msrdc.exe,MININT-IICQLJG.northamerica.corp.microsoft.com...
13092,2021-03-26T17:57:45.689000+00:00,9540.321,msrdc.exe,MININT-IICQLJG.northamerica.corp.microsoft.com...
11355,2021-03-31T22:33:55.816000+00:00,9118.532,msrdc.exe,MININT-IICQLJG.northamerica.corp.microsoft.com...
7737,2021-04-11T03:43:24.130000+00:00,9112.638,unknown,unknown
17694,2021-03-16T18:54:50.814000+00:00,8965.096,msrdc.exe,MININT-IICQLJG.northamerica.corp.microsoft.com...
13090,2021-03-26T20:36:49.731000+00:00,8717.533,msrdc.exe,MININT-IICQLJG.northamerica.corp.microsoft.com...
14130,2021-03-23T17:36:16.382000+00:00,8474.894,msrdc.exe,MININT-IICQLJG.northamerica.corp.microsoft.com...
11329,2021-04-01T01:48:43.010000+00:00,8368.512,msrdc.exe,MININT-IICQLJG.northamerica.corp.microsoft.com...


Time spent on LockApp.exe is time spent on the laptops homepage, therefore we will be removing it.

In [137]:
#removing rows with LockApp

activity_watch_final.drop(index=activity_watch_final[activity_watch_final['app'] == 'LockApp.exe'].index, inplace=True)



In [138]:
#removing rows with unknown

activity_watch_final.drop(index=activity_watch_final[activity_watch_final['app'] == 'unknown'].index, inplace=True)

In [139]:
#checking for different variations of the same name
activity_watch_final["app"].value_counts()

msedge.exe                                                        14985
explorer.exe                                                       2858
Code.exe                                                           1629
msrdc.exe                                                          1303
Code - Insiders.exe                                                1093
notepad.exe                                                         363
msrdcw.exe                                                          269
ApplicationFrameHost.exe                                            256
WindowsTerminal.exe                                                 239
ShellExperienceHost.exe                                             212
Zoom.exe                                                            181
Spotify.exe                                                         140
CredentialUIBroker.exe                                              137
SearchApp.exe                                                   

In [140]:
#sorting the values from the earliest date to the latest date
activity_watch_final.sort_values("timestamp_utc", ascending=True, inplace=True)

In [141]:
#resetting the index values
activity_watch_final.reset_index(drop=True, inplace=True)
activity_watch_final

,timestamp_utc,duration_seconds,app,title
0,2021-02-27T15:07:29.345000+00:00,52.882,msedge.exe,ActivityWatch/activitywatch: The best free and...
1,2021-02-27T15:08:23.428000+00:00,0.000,msedge.exe,https://github.com/ActivityWatch and 22 more p...
2,2021-02-27T15:08:24.630000+00:00,23.357,msedge.exe,ActivityWatch and 22 more pages - Personal - M...
3,2021-02-27T15:08:49.183000+00:00,19.718,msedge.exe,Sponsor @ActivityWatch on GitHub Sponsors and ...
4,2021-02-27T15:09:10.151000+00:00,4.846,SearchApp.exe,Search
...,...,...,...,...
24240,2021-04-18T14:15:15.525000+00:00,0.000,msedge.exe,localhost:7777/callback?code=AQAAFYrVKDTnt6XUV...
24241,2021-04-18T14:15:16.716000+00:00,0.000,msedge.exe,Oura API and 78 more pages - Personal - Micros...
24242,2021-04-18T14:15:17.916000+00:00,1.196,explorer.exe,
24243,2021-04-18T14:15:20.283000+00:00,0.000,aw-qt.exe,aw-qt


In [142]:
#For names that include "code" I will be changing it to "VSCode" to avoid double counting

for index in range(len(activity_watch_final["app"])):
    if "Code" in activity_watch_final["app"][index]:
        activity_watch_final["app"][index] = "VSCode"


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [143]:
#For names that include "spotify" I will be changing it to "Spotify" to avoid double counting

for index in range(len(activity_watch_final["app"])):
    if "Spotify" in activity_watch_final["app"][index] or "spotify" in activity_watch_final["app"][index]:
        activity_watch_final["app"][index] = "Spotify"


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [148]:
#For titles that are empty I am converting them to null values

for index in range(len(activity_watch_final["title"])):
    if len(activity_watch_final["title"][index]) == 0:
        activity_watch_final["title"][index] = np.nan

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [152]:
activity_watch_final[0:30]

,timestamp_utc,duration_seconds,app,title
0,2021-02-27T15:07:29.345000+00:00,52.882,msedge.exe,ActivityWatch/activitywatch: The best free and...
1,2021-02-27T15:08:23.428000+00:00,0.000,msedge.exe,https://github.com/ActivityWatch and 22 more p...
2,2021-02-27T15:08:24.630000+00:00,23.357,msedge.exe,ActivityWatch and 22 more pages - Personal - M...
3,2021-02-27T15:08:49.183000+00:00,19.718,msedge.exe,Sponsor @ActivityWatch on GitHub Sponsors and ...
4,2021-02-27T15:09:10.151000+00:00,4.846,SearchApp.exe,Search
5,2021-02-27T15:09:16.200000+00:00,13.051,explorer.exe,NaN
6,2021-02-27T15:09:30.308000+00:00,3.712,aw-qt.exe,aw-qt
7,2021-02-27T15:09:35.229000+00:00,0.104,msedge.exe,Untitled and 23 more pages - Personal - Micros...
8,2021-02-27T15:09:36.444000+00:00,0.089,msedge.exe,localhost:5600 and 23 more pages - Personal - ...
9,2021-02-27T15:09:37.642000+00:00,22.687,msedge.exe,ActivityWatch and 23 more pages - Personal - M...


In [119]:
#double checking for values that may be the same but have different names
activity_watch_final["app"].value_counts()

msedge.exe                      14985
explorer.exe                     2858
VSCode                           2750
msrdc.exe                        1303
notepad.exe                       363
msrdcw.exe                        269
ApplicationFrameHost.exe          256
WindowsTerminal.exe               239
ShellExperienceHost.exe           212
Zoom.exe                          181
Spotify                           143
CredentialUIBroker.exe            137
SearchApp.exe                     134
powershell.exe                     95
Taskmgr.exe                        53
brave.exe                          51
ScreenClippingHost.exe             39
EXCEL.EXE                          27
Telegram.exe                       21
dwm.exe                            18
WINWORD.EXE                        16
python.exe                         15
aw-qt.exe                          13
Teams.exe                          10
mmc.exe                             8
SystemPropertiesAdvanced.exe        7
SlideToShutD

In [169]:
activity_watch_final[activity_watch_final["app"]== "msedge.exe"].iloc[0:30]

,timestamp_utc,duration_seconds,app,title
0,2021-02-27T15:07:29.345000+00:00,52.882,msedge.exe,ActivityWatch/activitywatch: The best free and...
1,2021-02-27T15:08:23.428000+00:00,0.000,msedge.exe,https://github.com/ActivityWatch and 22 more p...
2,2021-02-27T15:08:24.630000+00:00,23.357,msedge.exe,ActivityWatch and 22 more pages - Personal - M...
3,2021-02-27T15:08:49.183000+00:00,19.718,msedge.exe,Sponsor @ActivityWatch on GitHub Sponsors and ...
7,2021-02-27T15:09:35.229000+00:00,0.104,msedge.exe,Untitled and 23 more pages - Personal - Micros...
8,2021-02-27T15:09:36.444000+00:00,0.089,msedge.exe,localhost:5600 and 23 more pages - Personal - ...
9,2021-02-27T15:09:37.642000+00:00,22.687,msedge.exe,ActivityWatch and 23 more pages - Personal - M...
10,2021-02-27T15:10:01.559000+00:00,0.031,msedge.exe,Features - ActivityWatch Forum and 23 more pag...
11,2021-02-27T15:10:02.769000+00:00,17.903,msedge.exe,Top Features topics - ActivityWatch Forum and ...
12,2021-02-27T15:10:21.865000+00:00,13.100,msedge.exe,ActivityWatch and 23 more pages - Personal - M...


In [15]:
#reading spotify data in

spotify = pd.read_csv("spotify_history_ore_2021_04_18.csv")
spotify.columns

Index(['timestamp', 'name', 'artists', 'uri', 'duration_ms', 'explicit'], dtype='object')

In [16]:
#reading google calendar data in

g_calendar = pd.read_csv("google_calendar_ore_2021_04_18.csv")
g_calendar.columns

Index(['kind', 'etag', 'id', 'status', 'htmlLink', 'created', 'updated',
       'summary', 'creator', 'organizer', 'start', 'end', 'recurringEventId',
       'originalStartTime', 'visibility', 'iCalUID', 'sequence', 'reminders',
       'eventType', 'description', 'colorId', 'transparency', 'location',
       'attendees', 'guestsCanInviteOthers', 'privateCopy', 'source',
       'hangoutLink', 'conferenceData', 'extendedProperties',
       'guestsCanModify', 'guestsCanSeeOtherGuests'],
      dtype='object')

In [17]:
#inspecting the data to identify any potential anomalies
g_calendar["created"].unique()

array(['2015-04-04T02:15:49.000Z', '2016-04-01T01:56:31.000Z',
       '2020-10-19T15:35:00.000Z', '2020-10-19T15:34:38.000Z',
       '2020-10-19T15:34:39.000Z', '2020-12-17T16:17:18.000Z',
       '2020-12-13T19:54:36.000Z', '2020-12-25T20:07:28.000Z',
       '2020-12-26T14:36:21.000Z', '2020-12-26T20:06:11.000Z',
       '2020-12-23T23:29:59.000Z', '2020-12-31T20:26:42.000Z',
       '2021-01-01T01:12:06.000Z', '2021-01-01T10:22:12.000Z',
       '2020-12-21T01:50:28.000Z', '2021-01-04T22:56:34.000Z',
       '2021-01-06T15:56:23.000Z', '2021-01-10T20:33:09.000Z',
       '2021-01-12T10:00:46.000Z', '2021-01-14T17:55:48.000Z',
       '2021-01-15T06:34:48.000Z', '2020-12-21T18:30:05.000Z',
       '2021-01-13T15:53:47.000Z', '2021-01-15T18:00:44.000Z',
       '2021-01-12T08:52:04.000Z', '2021-01-17T23:38:48.000Z',
       '2021-01-18T15:14:06.000Z', '2020-12-23T20:39:07.000Z',
       '2021-01-22T06:10:43.000Z', '2021-01-22T09:05:15.000Z',
       '2021-01-21T10:09:12.000Z', '2021-01-25T02:27:55

In [18]:
#inspecting the data to identify any potential anomalies
g_calendar["location"].value_counts()

Google Meet (instructions in description)                                                 59
https://meet.google.com/kgk-qgha-gai                                                       1
Amsterdam AMS                                                                              1
Tribeca Hotel, Accra                                                                       1
https://us02web.zoom.us/j/84101445821                                                      1
Schiphol Airport (Evert v/d Beekstraat 202, 1118 Amsterdam North Holland, Netherlands)     1
https://meet.google.com/vra-emhm-zoo                                                       1
Accra ACC                                                                                  1
https://zoom.us/j/92547005201                                                              1
Kotoka International Airport (Accra Accra, Ghana)                                          1
https://us02web.zoom.us/j/85090663120?pwd=NDhjRWt5UCtabGVkOTVxYm9lakN0

In [19]:
#inspecting the data to identify any potential anomalies
type(g_calendar["start"][0])

str

The columns start and end both contain dictionaries, so I will be separating them into dataframes then recombining.

In [20]:
#As discovered above, the dictionaries have been stored as strings. I will need to convert them into dictionaries so that I can split the columns.
g_calendar_start = g_calendar["start"].apply(ast.literal_eval)
g_calendar_start

0      {'dateTime': '2021-04-14T16:00:00-07:00', 'tim...
1                                 {'date': '2021-03-15'}
2              {'dateTime': '2021-01-11T04:00:00-08:00'}
3      {'dateTime': '2021-01-10T14:15:00-08:00', 'tim...
4      {'dateTime': '2021-01-11T04:00:00-08:00', 'tim...
                             ...                        
252            {'dateTime': '2021-04-17T12:15:00-07:00'}
253            {'dateTime': '2021-04-17T18:15:00-07:00'}
254            {'dateTime': '2021-04-17T20:30:00-07:00'}
255            {'dateTime': '2021-04-17T21:00:00-07:00'}
256            {'dateTime': '2021-04-17T21:00:00-07:00'}
Name: start, Length: 257, dtype: object

In [93]:
#The start times for google calendar events
g_calendar_start = g_calendar_start.apply(pd.Series)
g_calendar_start.rename(columns={"dateTime": "start_dateTime_utc"}, inplace=True)
g_calendar_start

,start_dateTime,timeZone,date
0,2021-04-14T16:00:00-07:00,Europe/Dublin,NaN
1,NaN,NaN,2021-03-15
2,2021-01-11T04:00:00-08:00,NaN,NaN
3,2021-01-10T14:15:00-08:00,Atlantic/Reykjavik,NaN
4,2021-01-11T04:00:00-08:00,Europe/Berlin,NaN
...,...,...,...
252,2021-04-17T12:15:00-07:00,NaN,NaN
253,2021-04-17T18:15:00-07:00,NaN,NaN
254,2021-04-17T20:30:00-07:00,NaN,NaN
255,2021-04-17T21:00:00-07:00,NaN,NaN


In [22]:
#splitting google calendar end date
g_calendar_end = g_calendar["end"].apply(ast.literal_eval)
g_calendar_end = g_calendar_end.apply(pd.Series)
g_calendar_end.rename(columns={"dateTime": "end_dateTime_utc"}, inplace=True)
g_calendar_end

,end_dateTime,timeZone,date
0,2021-04-14T17:00:00-07:00,Europe/Dublin,NaN
1,NaN,NaN,2021-03-16
2,2021-01-11T13:55:00-08:00,NaN,NaN
3,2021-01-10T20:50:00-08:00,Europe/Berlin,NaN
4,2021-01-11T13:55:00-08:00,America/Vancouver,NaN
...,...,...,...
252,2021-04-17T18:00:00-07:00,NaN,NaN
253,2021-04-17T20:15:00-07:00,NaN,NaN
254,2021-04-17T21:00:00-07:00,NaN,NaN
255,2021-04-17T22:15:00-07:00,NaN,NaN


from looking at these different datetimes, it looks as though it is in UTC minus X hours, depending on the timezone it is in.

In [23]:
#combining both the start and end columns into one dataframe

g_calendar_times_combined = g_calendar_start.join(g_calendar_end, lsuffix='_act', rsuffix='_res')
g_calendar_times_combined

,start_dateTime,timeZone_act,date_act,end_dateTime,timeZone_res,date_res
0,2021-04-14T16:00:00-07:00,Europe/Dublin,NaN,2021-04-14T17:00:00-07:00,Europe/Dublin,NaN
1,NaN,NaN,2021-03-15,NaN,NaN,2021-03-16
2,2021-01-11T04:00:00-08:00,NaN,NaN,2021-01-11T13:55:00-08:00,NaN,NaN
3,2021-01-10T14:15:00-08:00,Atlantic/Reykjavik,NaN,2021-01-10T20:50:00-08:00,Europe/Berlin,NaN
4,2021-01-11T04:00:00-08:00,Europe/Berlin,NaN,2021-01-11T13:55:00-08:00,America/Vancouver,NaN
...,...,...,...,...,...,...
252,2021-04-17T12:15:00-07:00,NaN,NaN,2021-04-17T18:00:00-07:00,NaN,NaN
253,2021-04-17T18:15:00-07:00,NaN,NaN,2021-04-17T20:15:00-07:00,NaN,NaN
254,2021-04-17T20:30:00-07:00,NaN,NaN,2021-04-17T21:00:00-07:00,NaN,NaN
255,2021-04-17T21:00:00-07:00,NaN,NaN,2021-04-17T22:15:00-07:00,NaN,NaN


In [121]:
#combining the calendar times dataframe with the original dataframe

g_calendar_final = g_calendar.join(g_calendar_times_combined, lsuffix='_act', rsuffix='_res')
g_calendar_final

,kind,etag,id,status,htmlLink,created,updated,summary,creator,organizer,...,conferenceData,extendedProperties,guestsCanModify,guestsCanSeeOtherGuests,start_dateTime,timeZone_act,date_act,end_dateTime,timeZone_res,date_res
0,calendar#event,"""2885397680780000""",7ui7kk61env04pff2r4oet7ba4_20210414T230000Z,confirmed,https://www.google.com/calendar/event?eid=N3Vp...,2015-04-04T02:15:49.000Z,2016-03-06T17:05:31.131Z,Jemimah's Birthday,"{'email': 'oreogundipe@gmail.com', 'displayNam...","{'email': 'oreogundipe@gmail.com', 'displayNam...",...,NaN,NaN,NaN,NaN,2021-04-14T16:00:00-07:00,Europe/Dublin,NaN,2021-04-14T17:00:00-07:00,Europe/Dublin,NaN
1,calendar#event,"""2918951584034000""",ccsj6c9oc9j62b9j70qjeb9k6gp3ab9pchi3cb9m6so36e...,confirmed,https://www.google.com/calendar/event?eid=Y2Nz...,2016-04-01T01:56:31.000Z,2016-04-01T01:56:32.052Z,It's your day Capt. OG,"{'email': 'oreogundipe@gmail.com', 'displayNam...","{'email': 'oreogundipe@gmail.com', 'displayNam...",...,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-15,NaN,NaN,2021-03-16
2,calendar#event,"""3211334650498000""",_6tlnaqrle5p6cpb4dhmj4phpehlmio9j6hlj4ord71jn0...,confirmed,https://www.google.com/calendar/event?eid=XzZ0...,2020-10-19T15:35:00.000Z,2020-12-17T13:49:10.095Z,Flight to Vancouver (KL 681),"{'email': 'oreogundipe@gmail.com', 'self': True}",{'email': 'unknownorganizer@calendar.google.com'},...,NaN,NaN,NaN,NaN,2021-01-11T04:00:00-08:00,NaN,NaN,2021-01-11T13:55:00-08:00,NaN,NaN
3,calendar#event,"""3211334719809000""",_60q30c1g60o30e1i60o4ac1g60rj8gpl88rj2c1h84s34...,confirmed,https://www.google.com/calendar/event?eid=XzYw...,2020-10-19T15:34:38.000Z,2020-12-17T13:50:40.936Z,KLM flight 590 to Amsterdam (N8JTST),"{'email': 'oreogundipe@gmail.com', 'self': True}","{'email': 'oreogundipe@gmail.com', 'self': True}",...,NaN,NaN,NaN,NaN,2021-01-10T14:15:00-08:00,Atlantic/Reykjavik,NaN,2021-01-10T20:50:00-08:00,Europe/Berlin,NaN
4,calendar#event,"""3211334720967000""",_60q30c1g60o30e1i60o4ac1g60rj8gpl88rj2c1h84s34...,confirmed,https://www.google.com/calendar/event?eid=XzYw...,2020-10-19T15:34:39.000Z,2020-12-17T13:50:41.515Z,KLM flight 681 to Vancouver (N8JTST),"{'email': 'oreogundipe@gmail.com', 'self': True}","{'email': 'oreogundipe@gmail.com', 'self': True}",...,NaN,NaN,NaN,NaN,2021-01-11T04:00:00-08:00,Europe/Berlin,NaN,2021-01-11T13:55:00-08:00,America/Vancouver,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,calendar#event,"""3237363132626000""",5mtl0s7s1l9f9d11584bmvvpih,confirmed,https://www.google.com/calendar/event?eid=NW10...,2021-04-17T17:46:06.000Z,2021-04-17T17:46:06.313Z,fusion - putting data together,"{'email': 'oreogundipe@gmail.com', 'self': True}","{'email': 'oreogundipe@gmail.com', 'self': True}",...,NaN,NaN,NaN,NaN,2021-04-17T12:15:00-07:00,NaN,NaN,2021-04-17T18:00:00-07:00,NaN,NaN
253,calendar#event,"""3237363179034000""",1961jk9r264751n3i683cdoi7g,confirmed,https://www.google.com/calendar/event?eid=MTk2...,2021-04-17T17:46:29.000Z,2021-04-17T17:46:29.517Z,go outside,"{'email': 'oreogundipe@gmail.com', 'self': True}","{'email': 'oreogundipe@gmail.com', 'self': True}",...,NaN,NaN,NaN,NaN,2021-04-17T18:15:00-07:00,NaN,NaN,2021-04-17T20:15:00-07:00,NaN,NaN
254,calendar#event,"""3237363211286000""",6o9tc9gtav6i52ond6tph8ujqq,confirmed,https://www.google.com/calendar/event?eid=Nm85...,2021-04-17T17:46:45.000Z,2021-04-17T17:46:45.643Z,somma - java homework,"{'email': 'oreogundipe@gmail.com', 'self': True}","{'email': 'oreogundipe@gmail.com', 'self': True}",...,NaN,NaN,NaN,NaN,2021-04-17T20:30:00-07:00,NaN,NaN,2021-04-17T21:00:00-07:00,NaN,NaN
255,calendar#event,"""3237363241070000""",6upb8fv4kkur6ugmunc8jsesm1,confirmed,https://www.google.com/calendar/event?eid=NnVw...,2021-04-17T17:47:00.000Z,2021-04-17T17:47:00.535Z,eat & nap,"{'email': 'oreogundipe@gmail.com', 'self': True}","{'email': 'oreogundipe@gmail.com', 'self': True}",...,NaN,NaN,NaN,NaN,2021-04-17T21:00:00-07:00,NaN,NaN,2021-04-17T22:15:00-07:00,NaN,NaN


In [155]:
g_calendar_final["timeZone_act"].value_counts()

America/Vancouver     112
Africa/Algiers         14
America/New_York        5
Europe/London           2
Europe/Berlin           1
Europe/Dublin           1
Atlantic/Reykjavik      1
Name: timeZone_act, dtype: int64

In [25]:
#here I will be dropping the irrelevant columns

Based on the results above I will only be keeping the following columns as they provide us with the most information:

- meeting start time (g_calendar_start)
- meeting end time (g_clanedar_end)
- timezone_act
- created
- summary
- description

In [26]:
#reading oura file in

oura_file = open('oura_data_ore_2021_04_18.json')
oura_json = oura_file.read()
oura_data = json.loads(oura_json)


In [27]:
#splitting oura_data into first dataframe which just has activity data. I will be combining it with other oura dataframes on the summary_date column as that column is in all dataframes

oura_activity = pd.DataFrame.from_records(oura_data["activity"])
oura_activity
oura_activity["summary_date"].describe()

count             84
unique            84
top       2021-03-14
freq               1
Name: summary_date, dtype: object

In [28]:
#extracting restufl periods into it's own dataframe

oura_restful_periods = pd.DataFrame.from_records(oura_data["restful_periods"])
oura_restful_periods
oura_restful_periods["summary_date"].describe()

count             52
unique            39
top       2021-01-31
freq               3
Name: summary_date, dtype: object

In [29]:
#extracting readiness into it's own dataframe

oura_readiness = pd.DataFrame.from_records(oura_data["readiness"])
type(oura_readiness["summary_date"][5])
oura_readiness

,period_id,rest_mode_state,score,score_activity_balance,score_hrv_balance,score_previous_day,score_previous_night,score_recovery_index,score_resting_hr,score_sleep_balance,score_temperature,summary_date
0,0,0,87,0,0,0,71,100,95,0,99,2021-01-21
1,1,0,73,0,0,90,62,48,90,0,100,2021-01-22
2,1,0,87,0,0,89,82,100,93,80,90,2021-01-23
3,2,0,81,92,0,93,70,81,95,70,98,2021-01-24
4,0,0,51,59,0,74,32,7,82,50,97,2021-01-25
...,...,...,...,...,...,...,...,...,...,...,...,...
77,1,0,47,36,70,87,39,57,50,19,92,2021-04-13
78,0,0,44,31,64,51,64,64,27,25,88,2021-04-14
79,1,0,54,40,66,80,46,74,76,25,100,2021-04-15
80,0,0,46,52,69,68,26,41,68,21,90,2021-04-16


In [30]:
#extracting sleep into it's own dataframe

oura_sleep = pd.DataFrame.from_records(oura_data["sleep"])
oura_sleep["summary_date"][3]

'2021-01-24'

In [31]:
#left joining datarame on the summary column first
oura_combined = oura_activity.join(oura_restful_periods, lsuffix='_act', rsuffix='_res')
oura_combined

,average_met,cal_active,cal_total,class_5min,daily_movement,day_end,day_start,high,inactive,inactivity_alerts,...,bedtime_end,bedtime_start,breath_average,duration,period_id,summary_date_res,timezone_res,hr_average,hr_lowest,rmssd
0,0.93750,23,1942,0000000000000000000000000000000000000000000000...,461,2021-01-21T03:59:59-08:00,2021-01-20T04:00:00-08:00,0,141,1,...,2021-01-21T07:36:24-08:00,2021-01-21T07:23:24-08:00,15.250,780.0,1.0,2021-01-21,-480.0,NaN,NaN,NaN
1,1.34375,234,2311,2232332222222221222222322222333332322223311221...,4151,2021-01-22T03:59:59-08:00,2021-01-21T04:00:00-08:00,1,792,1,...,2021-01-23T13:25:29-08:00,2021-01-23T13:09:29-08:00,15.125,960.0,3.0,2021-01-23,-480.0,59.00,58.0,64.0
2,1.34375,290,2350,1211222221111111122212211222222222222224434333...,5337,2021-01-23T03:59:59-08:00,2021-01-22T04:00:00-08:00,0,612,0,...,2021-01-23T14:50:29-08:00,2021-01-23T14:39:29-08:00,15.125,660.0,4.0,2021-01-23,-480.0,60.00,60.0,NaN
3,1.15625,96,2048,2232222211122211111111111111111111112211111111...,1832,2021-01-24T03:59:59-08:00,2021-01-23T04:00:00-08:00,0,517,1,...,2021-01-28T15:34:12-08:00,2021-01-28T15:06:12-08:00,15.250,1680.0,1.0,2021-01-28,-480.0,85.67,83.0,29.0
4,1.68750,707,2894,1111111111111111111111221212000000000221111112...,13257,2021-01-25T03:59:59-08:00,2021-01-24T04:00:00-08:00,2,703,0,...,2021-01-30T13:01:59-08:00,2021-01-30T12:40:59-08:00,14.875,1260.0,1.0,2021-01-30,-480.0,65.75,64.0,63.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,1.65625,721,2859,1111112112111112222222222222222323222222212211...,12814,2021-04-14T03:59:59-07:00,2021-04-13T04:00:00-07:00,20,710,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,1.40625,301,2400,1111111111111111111112111111111111111121121121...,5541,2021-04-15T03:59:59-07:00,2021-04-14T04:00:00-07:00,0,774,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,1.37500,193,2324,1111111111111112111112221222222222233333222222...,3148,2021-04-16T03:59:59-07:00,2021-04-15T04:00:00-07:00,1,1062,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,1.50000,550,2609,1111111111111111122111111111111111111221122211...,10674,2021-04-17T03:59:59-07:00,2021-04-16T04:00:00-07:00,2,451,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
#combining the readiness dataframe to the above combined dataframe
oura_combined_1 = oura_combined.join(oura_readiness, lsuffix='_act_1', rsuffix='_res_1')
oura_combined_1

,average_met,cal_active,cal_total,class_5min,daily_movement,day_end,day_start,high,inactive,inactivity_alerts,...,score_res_1,score_activity_balance,score_hrv_balance,score_previous_day,score_previous_night,score_recovery_index,score_resting_hr,score_sleep_balance,score_temperature,summary_date
0,0.93750,23,1942,0000000000000000000000000000000000000000000000...,461,2021-01-21T03:59:59-08:00,2021-01-20T04:00:00-08:00,0,141,1,...,87.0,0.0,0.0,0.0,71.0,100.0,95.0,0.0,99.0,2021-01-21
1,1.34375,234,2311,2232332222222221222222322222333332322223311221...,4151,2021-01-22T03:59:59-08:00,2021-01-21T04:00:00-08:00,1,792,1,...,73.0,0.0,0.0,90.0,62.0,48.0,90.0,0.0,100.0,2021-01-22
2,1.34375,290,2350,1211222221111111122212211222222222222224434333...,5337,2021-01-23T03:59:59-08:00,2021-01-22T04:00:00-08:00,0,612,0,...,87.0,0.0,0.0,89.0,82.0,100.0,93.0,80.0,90.0,2021-01-23
3,1.15625,96,2048,2232222211122211111111111111111111112211111111...,1832,2021-01-24T03:59:59-08:00,2021-01-23T04:00:00-08:00,0,517,1,...,81.0,92.0,0.0,93.0,70.0,81.0,95.0,70.0,98.0,2021-01-24
4,1.68750,707,2894,1111111111111111111111221212000000000221111112...,13257,2021-01-25T03:59:59-08:00,2021-01-24T04:00:00-08:00,2,703,0,...,51.0,59.0,0.0,74.0,32.0,7.0,82.0,50.0,97.0,2021-01-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,1.65625,721,2859,1111112112111112222222222222222323222222212211...,12814,2021-04-14T03:59:59-07:00,2021-04-13T04:00:00-07:00,20,710,1,...,54.0,40.0,66.0,80.0,46.0,74.0,76.0,25.0,100.0,2021-04-15
80,1.40625,301,2400,1111111111111111111112111111111111111121121121...,5541,2021-04-15T03:59:59-07:00,2021-04-14T04:00:00-07:00,0,774,1,...,46.0,52.0,69.0,68.0,26.0,41.0,68.0,21.0,90.0,2021-04-16
81,1.37500,193,2324,1111111111111112111112221222222222233333222222...,3148,2021-04-16T03:59:59-07:00,2021-04-15T04:00:00-07:00,1,1062,3,...,35.0,51.0,57.0,86.0,40.0,23.0,1.0,17.0,96.0,2021-04-17
82,1.50000,550,2609,1111111111111111122111111111111111111221122211...,10674,2021-04-17T03:59:59-07:00,2021-04-16T04:00:00-07:00,2,451,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
#combining the readiness dataframe to the above combined dataframe, to make the full oura dataframe
oura_test_combined_final = oura_combined_1.join(oura_sleep, lsuffix='_act_1', rsuffix='_res_1')
oura_test_combined_final

,average_met,cal_active,cal_total,class_5min,daily_movement,day_end,day_start,high,inactive,inactivity_alerts,...,score_efficiency,score_latency,score_rem,score_total,summary_date_res_1,temperature_delta,temperature_deviation,timezone,total_res_1,temperature_trend_deviation
0,0.93750,23,1942,0000000000000000000000000000000000000000000000...,461,2021-01-21T03:59:59-08:00,2021-01-20T04:00:00-08:00,0,141,1,...,86.0,91.0,63.0,84.0,2021-01-21,-0.15,-0.15,-480.0,28830.0,NaN
1,1.34375,234,2311,2232332222222221222222322222333332322223311221...,4151,2021-01-22T03:59:59-08:00,2021-01-21T04:00:00-08:00,1,792,1,...,83.0,67.0,39.0,55.0,2021-01-22,-0.10,-0.10,-480.0,21420.0,0.01
2,1.34375,290,2350,1211222221111111122212211222222222222224434333...,5337,2021-01-23T03:59:59-08:00,2021-01-22T04:00:00-08:00,0,612,0,...,69.0,97.0,86.0,82.0,2021-01-23,-0.32,-0.32,-480.0,28410.0,0.01
3,1.15625,96,2048,2232222211122211111111111111111111112211111111...,1832,2021-01-24T03:59:59-08:00,2021-01-23T04:00:00-08:00,0,517,1,...,86.0,86.0,66.0,61.0,2021-01-24,-0.26,-0.26,-480.0,22890.0,0.02
4,1.68750,707,2894,1111111111111111111111221212000000000221111112...,13257,2021-01-25T03:59:59-08:00,2021-01-24T04:00:00-08:00,2,703,0,...,76.0,91.0,72.0,69.0,2021-01-27,0.16,0.16,-480.0,25020.0,0.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,1.65625,721,2859,1111112112111112222222222222222323222222212211...,12814,2021-04-14T03:59:59-07:00,2021-04-13T04:00:00-07:00,20,710,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,1.40625,301,2400,1111111111111111111112111111111111111121121121...,5541,2021-04-15T03:59:59-07:00,2021-04-14T04:00:00-07:00,0,774,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,1.37500,193,2324,1111111111111112111112221222222222233333222222...,3148,2021-04-16T03:59:59-07:00,2021-04-15T04:00:00-07:00,1,1062,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,1.50000,550,2609,1111111111111111122111111111111111111221122211...,10674,2021-04-17T03:59:59-07:00,2021-04-16T04:00:00-07:00,2,451,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
#inspecting what columns to drop
oura_test_combined_final.columns

Index(['average_met', 'cal_active', 'cal_total', 'class_5min',
       'daily_movement', 'day_end', 'day_start', 'high', 'inactive',
       'inactivity_alerts', 'low', 'medium', 'met_1min', 'met_min_high',
       'met_min_inactive', 'met_min_low', 'met_min_medium', 'non_wear', 'rest',
       'rest_mode_state_act_1', 'score_act_1', 'score_meet_daily_targets',
       'score_move_every_hour', 'score_recovery_time', 'score_stay_active',
       'score_training_frequency', 'score_training_volume', 'steps',
       'summary_date_act', 'target_calories', 'target_km', 'target_miles',
       'timezone_act', 'to_target_km', 'to_target_miles', 'total_act_1',
       'bedtime_end_act_1', 'bedtime_start_act_1', 'breath_average_act_1',
       'duration_act_1', 'period_id_act_1', 'summary_date_res', 'timezone_res',
       'hr_average_act_1', 'hr_lowest_act_1', 'rmssd_act_1', 'period_id_res_1',
       'rest_mode_state_res_1', 'score_res_1', 'score_activity_balance',
       'score_hrv_balance', 'score_prev

In [156]:
#checking for numerical anomalies

oura_test_combined_final["inactive"].describe()

count      84.000000
mean      623.785714
std       182.630846
min       141.000000
25%       516.500000
50%       648.500000
75%       745.250000
max      1062.000000
Name: inactive, dtype: float64

In [ ]:
#renaming columsn with the correct unit of time
oura_test_combined_final.rename(columns={"inactive":"inactive_minutes"}, inplace=True)
oura_test_combined_final.rename(columns={"non_wear":"non_wear_minutes"}, inplace=True)
oura_test_combined_final.rename(columns={"rest":"rest_minutes"}, inplace=True)
oura_test_combined_final.rename(columns={"low":"low_minutes"}, inplace=True)
oura_test_combined_final.rename(columns={"medium":"medium_minutes"}, inplace=True)
oura_test_combined_final.rename(columns={"high":"high_minutes"}, inplace=True)

In [ ]:
#checking for numerical anomalies in the

oura_test_combined_final[oura_test_combined_final["daily_movement"] > 6000]

In [62]:
twitter_tweets = pd.read_json("ore_tweets_content.json")
twitter_tweets

,tweet
0,"{'retweeted': False, 'source': '<a href=""http:..."
1,"{'retweeted': False, 'source': '<a href=""http:..."
2,"{'retweeted': False, 'source': '<a href=""http:..."
3,"{'retweeted': False, 'source': '<a href=""http:..."
4,"{'retweeted': False, 'source': '<a href=""http:..."
...,...
2699,"{'retweeted': False, 'source': '<a href=""http:..."
2700,"{'retweeted': False, 'source': '<a href=""http:..."
2701,"{'retweeted': False, 'source': '<a href=""http:..."
2702,"{'retweeted': False, 'source': '<a href=""http:..."


In [63]:
twitter_likes = pd.read_json("ore_twitter_likes.json")
twitter_likes

,like
0,"{'tweetId': '1382005671672082439', 'fullText':..."
1,"{'tweetId': '1384147599670747138', 'fullText':..."
2,"{'tweetId': '1381859466329477124', 'fullText':..."
3,"{'tweetId': '1384016658226810886', 'fullText':..."
4,"{'tweetId': '1384105155163942917', 'fullText':..."
...,...
8508,"{'tweetId': '1271045278146199553', 'fullText':..."
8509,"{'tweetId': '1271158491038789632', 'fullText':..."
8510,"{'tweetId': '1271271067630239744', 'fullText':..."
8511,"{'tweetId': '1271144500706992128', 'fullText':..."
